<h1><center>Laboratorio N° 03: Pivoting y Grouping Sets

<h2>Parte 01. Escribir consultas que utilizan PIVOT.

1.1. Abrir la base de datos TSQL.

In [18]:
USE TSQL;
GO

Commands completed successfully.

Total execution time: 00:00:00

1.2. Crear un vista de categorías de productos, sus cantidaddes por año.

In [19]:
CREATE VIEW Production.VistaCategoriasPorAño AS
SELECT 
    c.CategoryName,
    YEAR(o.OrderDate) AS Año,
    SUM(od.qty) AS TotalCantidad
FROM 
    Production.Categories c
    INNER JOIN Production.Products p ON c.CategoryID = p.CategoryID
    INNER JOIN Sales.OrderDetails od ON p.ProductID = od.ProductID
    INNER JOIN Sales.Orders o ON od.OrderID = o.OrderID
GROUP BY 
    c.CategoryName, YEAR(o.OrderDate);

Commands completed successfully.

Total execution time: 00:00:00.033

1.3. Probar la vista creada.

In [20]:
SELECT * FROM Production.VistaCategoriasPorAño;

(24 rows affected)

Total execution time: 00:00:00.009

CategoryName,Año,TotalCantidad
Meat/Poultry,2006,950
Seafood,2008,2716
Dairy Products,2007,4374
Grains/Cereals,2008,1377
Confections,2007,4137
Produce,2008,858
Beverages,2006,1842
Condiments,2008,1441
Dairy Products,2008,2689
Seafood,2007,3679


1.4. Utilizar PIVOT entre categorías y años de ordenes.

In [10]:
SELECT CategoryName, [2018], [2019], [2020], [2021], [2022]
FROM
(
    SELECT 
        CategoryName, 
        Año, 
        TotalCantidad
    FROM 
        Production.VistaCategoriasPorAño
) AS SourceTable
PIVOT
(
    SUM(TotalCantidad)
    FOR Año IN ([2018], [2019], [2020], [2021], [2022])
) AS PivotTable;

(8 rows affected)

Total execution time: 00:00:00.013

CategoryName,2018,2019,2020,2021,2022
Beverages,NULL,NULL,NULL,NULL,NULL
Condiments,NULL,NULL,NULL,NULL,NULL
Confections,NULL,NULL,NULL,NULL,NULL
Dairy Products,NULL,NULL,NULL,NULL,NULL
Grains/Cereals,NULL,NULL,NULL,NULL,NULL
Meat/Poultry,NULL,NULL,NULL,NULL,NULL
Produce,NULL,NULL,NULL,NULL,NULL
Seafood,NULL,NULL,NULL,NULL,NULL


1.5. Para utilizar UNPIVOT, crear una tabla con el resultado de la consulta con PIVOT anterior.

In [11]:
SELECT CategoryName, [2018], [2019], [2020], [2021], [2022]
INTO dbo.ResultadoPivot
FROM
(
    SELECT 
        CategoryName, 
        Año, 
        TotalCantidad
    FROM 
        Production.VistaCategoriasPorAño
) AS SourceTable
PIVOT
(
    SUM(TotalCantidad)
    FOR Año IN ([2018], [2019], [2020], [2021], [2022])
) AS PivotTable;

(8 rows affected)

Total execution time: 00:00:00.015

1.6. Probrar la tabla generada

In [12]:
SELECT * FROM dbo.ResultadoPivot;

(8 rows affected)

Total execution time: 00:00:00.003

CategoryName,2018,2019,2020,2021,2022
Beverages,NULL,NULL,NULL,NULL,NULL
Condiments,NULL,NULL,NULL,NULL,NULL
Confections,NULL,NULL,NULL,NULL,NULL
Dairy Products,NULL,NULL,NULL,NULL,NULL
Grains/Cereals,NULL,NULL,NULL,NULL,NULL
Meat/Poultry,NULL,NULL,NULL,NULL,NULL
Produce,NULL,NULL,NULL,NULL,NULL
Seafood,NULL,NULL,NULL,NULL,NULL


1.7. Utilizar UNPIVOT.

In [13]:
SELECT CategoryName, Año, TotalCantidad
FROM
    (SELECT CategoryName, [2018], [2019], [2020], [2021], [2022]
     FROM dbo.ResultadoPivot) p
UNPIVOT
    (TotalCantidad FOR Año IN ([2018], [2019], [2020], [2021], [2022]))
    AS unpvt;

(0 rows affected)

Total execution time: 00:00:00.013

CategoryName,Año,TotalCantidad


1.8. Limpiar los cambios realizados.


In [16]:
-- Eliminar la tabla resultado del PIVOT si existe
IF OBJECT_ID('dbo.ResultadoPivot', 'U') IS NOT NULL
    DROP TABLE dbo.ResultadoPivot;

-- Eliminar la vista de categorías por año si existe
IF OBJECT_ID('Production.VistaCategoriasPorAño', 'V') IS NOT NULL
    DROP VIEW Production.VistaCategoriasPorAño;

Commands completed successfully.

Total execution time: 00:00:00

<h2>Parte 02. Escribir consultas que utilizan UNPIVOT.

2.1. Ejecutar la siguiente vistas

In [21]:
SELECT * FROM Production.VistaCategoriasPorAño;

(24 rows affected)

Total execution time: 00:00:00.006

CategoryName,Año,TotalCantidad
Meat/Poultry,2006,950
Seafood,2008,2716
Dairy Products,2007,4374
Grains/Cereals,2008,1377
Confections,2007,4137
Produce,2008,858
Beverages,2006,1842
Condiments,2008,1441
Dairy Products,2008,2689
Seafood,2007,3679


2.2. Elaborar una consulta sin utilizar GROUPING SETS.

In [22]:
-- 1. Total por Categoría y Año
SELECT 
    c.CategoryName,
    YEAR(o.OrderDate) AS Año,
    SUM(od.qty) AS TotalCantidad
FROM 
    Production.Categories c
    INNER JOIN Production.Products p ON c.CategoryID = p.CategoryID
    INNER JOIN Sales.OrderDetails od ON p.ProductID = od.ProductID
    INNER JOIN Sales.Orders o ON od.OrderID = o.OrderID
GROUP BY 
    c.CategoryName, YEAR(o.OrderDate)

UNION ALL

-- 2. Total por Año (todas las categorías)
SELECT 
    NULL AS CategoryName,
    YEAR(o.OrderDate) AS Año,
    SUM(od.qty) AS TotalCantidad
FROM 
    Production.Categories c
    INNER JOIN Production.Products p ON c.CategoryID = p.CategoryID
    INNER JOIN Sales.OrderDetails od ON p.ProductID = od.ProductID
    INNER JOIN Sales.Orders o ON od.OrderID = o.OrderID
GROUP BY 
    YEAR(o.OrderDate)

UNION ALL

-- 3. Total general (todas las categorías y todos los años)
SELECT 
    NULL AS CategoryName,
    NULL AS Año,
    SUM(od.qty) AS TotalCantidad
FROM 
    Production.Categories c
    INNER JOIN Production.Products p ON c.CategoryID = p.CategoryID
    INNER JOIN Sales.OrderDetails od ON p.ProductID = od.ProductID
    INNER JOIN Sales.Orders o ON od.OrderID = o.OrderID;

(28 rows affected)

Total execution time: 00:00:00.029

CategoryName,Año,TotalCantidad
Meat/Poultry,2006,950
Seafood,2008,2716
Dairy Products,2007,4374
Grains/Cereals,2008,1377
Confections,2007,4137
Produce,2008,858
Beverages,2006,1842
Condiments,2008,1441
Dairy Products,2008,2689
Seafood,2007,3679


2.3. Consultar con GROUPING SETS.

In [23]:
SELECT 
    c.CategoryName,
    YEAR(o.OrderDate) AS Año,
    SUM(od.qty) AS TotalCantidad
FROM 
    Production.Categories c
    INNER JOIN Production.Products p ON c.CategoryID = p.CategoryID
    INNER JOIN Sales.OrderDetails od ON p.ProductID = od.ProductID
    INNER JOIN Sales.Orders o ON od.OrderID = o.OrderID
GROUP BY 
    GROUPING SETS (
        (c.CategoryName, YEAR(o.OrderDate)),   -- Total por categoría y año
        (c.CategoryName),                      -- Total por categoría
        (YEAR(o.OrderDate)),                   -- Total por año
        ()                                     -- Gran total general
    );

(36 rows affected)

Total execution time: 00:00:00.032

CategoryName,Año,TotalCantidad
Beverages,2006,1842
Condiments,2006,962
Confections,2006,1357
Dairy Products,2006,2086
Grains/Cereals,2006,549
Meat/Poultry,2006,950
Produce,2006,549
Seafood,2006,1286
NULL,2006,9581
Beverages,2007,3996


2.4. Consultar con CUBE.

In [24]:
SELECT 
    c.CategoryName,
    YEAR(o.OrderDate) AS Año,
    SUM(od.qty) AS TotalCantidad
FROM 
    Production.Categories c
    INNER JOIN Production.Products p ON c.CategoryID = p.CategoryID
    INNER JOIN Sales.OrderDetails od ON p.ProductID = od.ProductID
    INNER JOIN Sales.Orders o ON od.OrderID = o.OrderID
GROUP BY 
    CUBE (c.CategoryName, YEAR(o.OrderDate));

(36 rows affected)

Total execution time: 00:00:00.015

CategoryName,Año,TotalCantidad
Beverages,2006,1842
Condiments,2006,962
Confections,2006,1357
Dairy Products,2006,2086
Grains/Cereals,2006,549
Meat/Poultry,2006,950
Produce,2006,549
Seafood,2006,1286
NULL,2006,9581
Beverages,2007,3996


2.5. Consultar con ROLLUP

In [25]:
SELECT 
    c.CategoryName,
    YEAR(o.OrderDate) AS Año,
    SUM(od.qty) AS TotalCantidad
FROM 
    Production.Categories c
    INNER JOIN Production.Products p ON c.CategoryID = p.CategoryID
    INNER JOIN Sales.OrderDetails od ON p.ProductID = od.ProductID
    INNER JOIN Sales.Orders o ON od.OrderID = o.OrderID
GROUP BY 
    ROLLUP (YEAR(o.OrderDate), c.CategoryName);


(28 rows affected)

Total execution time: 00:00:00.020

CategoryName,Año,TotalCantidad
Beverages,2006,1842
Condiments,2006,962
Confections,2006,1357
Dairy Products,2006,2086
Grains/Cereals,2006,549
Meat/Poultry,2006,950
Produce,2006,549
Seafood,2006,1286
NULL,2006,9581
Beverages,2007,3996


2.6. Utilizar Grouping_ID.

In [26]:
SELECT 
    c.CategoryName,
    YEAR(o.OrderDate) AS Año,
    SUM(od.qty) AS TotalCantidad,
    GROUPING_ID(YEAR(o.OrderDate), c.CategoryName) AS NivelAgrupacion
FROM 
    Production.Categories c
    INNER JOIN Production.Products p ON c.CategoryID = p.CategoryID
    INNER JOIN Sales.OrderDetails od ON p.ProductID = od.ProductID
    INNER JOIN Sales.Orders o ON od.OrderID = o.OrderID
GROUP BY 
    ROLLUP (YEAR(o.OrderDate), c.CategoryName);


(28 rows affected)

Total execution time: 00:00:00.016

CategoryName,Año,TotalCantidad,NivelAgrupacion
Beverages,2006,1842,0
Condiments,2006,962,0
Confections,2006,1357,0
Dairy Products,2006,2086,0
Grains/Cereals,2006,549,0
Meat/Poultry,2006,950,0
Produce,2006,549,0
Seafood,2006,1286,0
NULL,2006,9581,1
Beverages,2007,3996,0


2.7. Finalmente limpiar los cambios

In [27]:
-- Eliminar la tabla creada para el pivot
IF OBJECT_ID('dbo.ResultadoPivot', 'U') IS NOT NULL
    DROP TABLE dbo.ResultadoPivot;

-- Eliminar la vista creada para las categorías por año
IF OBJECT_ID('Production.VistaCategoriasPorAño', 'V') IS NOT NULL
    DROP VIEW Production.VistaCategoriasPorAño;


Commands completed successfully.

Total execution time: 00:00:00.003

<h2>Ejercicio 03: Escribir consultas que usan las clausulas GROUPING SETS, CUBE y ROLLUP

3.1. Escribir una sentencia SELECT contra la tabla Sales.Customers y traer las columnas country, city
y una columna calculada noofcustomers como una cantidad de clientes. Traer multiples grouping
sets basados en las columnas country y city, la columna country, la columna city, y una columna con un grouping set vacio.

In [28]:
SELECT 
    country, 
    city, 
    COUNT(*) AS noofcustomers
FROM 
    Sales.Customers
GROUP BY 
    GROUPING SETS (
        (country, city),   -- Total de clientes por país y ciudad
        (country),         -- Total de clientes por país
        (city),            -- Total de clientes por ciudad
        ()                 -- Gran total (todos los clientes)
    );


(160 rows affected)

Total execution time: 00:00:00.017

country,city,noofcustomers
Germany,Aachen,1
NULL,Aachen,1
USA,Albuquerque,1
NULL,Albuquerque,1
USA,Anchorage,1
NULL,Anchorage,1
Denmark,Århus,1
NULL,Århus,1
Spain,Barcelona,1
NULL,Barcelona,1


3.2. Escribir una sentencia SELECT contra la vista Sales.OrderValues y traer las columnas:
<li>Año de la columna orderdate como orderyear.
<li>Mes de la columna orderdate como ordermonth.
<li>Día de la columna orderdate como orderday.
<li>Valor de Total de Ventas utilizando la columna val como salesvalue.
<li>Devolver todas los posibles  grouping sets basados en las columnas orderyear, ordermonth, y orderday.

In [29]:
SELECT
    YEAR(orderdate) AS orderyear,
    MONTH(orderdate) AS ordermonth,
    DAY(orderdate) AS orderday,
    SUM(val) AS salesvalue
FROM
    Sales.OrderValues
GROUP BY
    GROUPING SETS (
        (YEAR(orderdate), MONTH(orderdate), DAY(orderdate)),  -- año, mes, día
        (YEAR(orderdate), MONTH(orderdate)),                  -- año, mes
        (YEAR(orderdate)),                                    -- año
        ()                                                    -- total general
    );


(507 rows affected)

Total execution time: 00:00:00.019

orderyear,ordermonth,orderday,salesvalue
2006,7,4,440.00
2006,7,5,1863.40
2006,7,8,2206.66
2006,7,9,3597.90
2006,7,10,1444.80
2006,7,11,556.62
2006,7,12,2490.50
2006,7,15,517.80
2006,7,16,1119.90
2006,7,17,1614.88


3.3. Copiar la consulta previa y modificarla para utilizar la clausula ROLLUP en vez de CUBE.

In [30]:
SELECT
    YEAR(orderdate) AS orderyear,
    MONTH(orderdate) AS ordermonth,
    DAY(orderdate) AS orderday,
    SUM(val) AS salesvalue
FROM
    Sales.OrderValues
GROUP BY
    ROLLUP (
        YEAR(orderdate),
        MONTH(orderdate),
        DAY(orderdate)
    );


(507 rows affected)

Total execution time: 00:00:00.018

orderyear,ordermonth,orderday,salesvalue
2006,7,4,440.00
2006,7,5,1863.40
2006,7,8,2206.66
2006,7,9,3597.90
2006,7,10,1444.80
2006,7,11,556.62
2006,7,12,2490.50
2006,7,15,517.80
2006,7,16,1119.90
2006,7,17,1614.88


3.4. Escribir una sentencia SELECT contra la vista Sales.OrderValues y traer las columnas:
<li>Columna Calculada con el alias groupid (usar la función GROUPING_ID con los campos order year y order month como parametros).
<li>Año de la columna orderdate como orderyear.
<li>Mes de la columna orderdate como ordermonth.
<li>Valor Total de Ventas utilizando la columna val como salesvalue.
<li>Desde Año y mes month forman una jerarquia, retornar todos los grouping sets interesantes basados en las columnas orderyear y ordermonth y ordenar el resultado por groupid, orderyear, and ordermonth

In [31]:
SELECT
    GROUPING_ID(YEAR(orderdate), MONTH(orderdate)) AS groupid,
    YEAR(orderdate) AS orderyear,
    MONTH(orderdate) AS ordermonth,
    SUM(val) AS salesvalue
FROM
    Sales.OrderValues
GROUP BY
    GROUPING SETS (
        (YEAR(orderdate), MONTH(orderdate)), -- Totales por año y mes
        (YEAR(orderdate)),                   -- Totales por año
        ()                                   -- Gran total
    )
ORDER BY
    groupid,
    orderyear,
    ordermonth;


(27 rows affected)

Total execution time: 00:00:00.016

groupid,orderyear,ordermonth,salesvalue
0,2006,7,27861.90
0,2006,8,25485.28
0,2006,9,26381.40
0,2006,10,37515.73
0,2006,11,45600.05
0,2006,12,45239.63
0,2007,1,61258.08
0,2007,2,38483.64
0,2007,3,38547.23
0,2007,4,53032.95
